In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder

# Download the necessary resources for nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.model_selection import train_test_split
from textblob import TextBlob, Word
from wordcloud import WordCloud,STOPWORDS
from subprocess import check_output
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer

ModuleNotFoundError: No module named 'textblob'

In [17]:
!pip install textblob



  Using cached textblob-0.19.0-py3-none-any.whl.metadata (4.4 kB)
Using cached textblob-0.19.0-py3-none-any.whl (624 kB)


In [5]:
df=pd.read_csv("./Sentiment.csv")
df.head()
df['sentiment'].value_counts()
df=df[["text","sentiment"]]

FileNotFoundError: [Errno 2] No such file or directory: './Sentiment.csv'

In [18]:
import os
print(os.getcwd())

C:\Users\hp\Desktop\Performance-Comparision-of-Various-Optimizers-for-Sentiment-Analysis


In [19]:
# df = df[df.sentiment != "Neutral"]
# df['sentiment']= pd.get_dummies(df['sentiment'], drop_first = True)
# df = df.reset_index(drop=True)
# df.head()

In [7]:
def cleantxt(text):
    text= re.sub(r'@[A-Za-z0-9]+', '',text)# removed @mentions
    text= re.sub(r'#', '',text)# removed # symbol
    text = re.sub(r'RT[\s]+', '',text)# rmoved RT
    text = re.sub(r'https?:\/\/\s+', '',text)# removed the hyperlink
    text = re.sub(r':+', '',text)# removed : symbol
    text = re.sub(r'--+', '',text)# removed : symbol
    text = re.sub(r'http', '',text)
    return text
df["text"] = df["text"].apply(cleantxt)
df["text"] = df["text"].apply(lambda el: el.replace("RT ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("#GOPDebate ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("#GOPDebates ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("#GOPdebate ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("#gopdebate ", ""))
df["text"] = df["text"].str.replace("@.+: ","")
df["text"] = df["text"].apply(lambda el: el.replace("w/ ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("#TedCruz ", "Ted Cruz "))
df["text"] = df["text"].apply(lambda el: el.replace("@realDonaldTrump ", "Donald Trump "))
df["text"] = df["text"].str.replace(".+:\/.+t.co\/.+", "")
df["text"] = df["text"].str.replace("(#.+ )|(#.+)", "")
df["text"] = df["text"].str.replace("@.+ ", "")
# df["text"] = df["text"].apply(lambda el: replace_emoj(el))
df["text"] = df["text"].str.replace("[^\w\s]","")
df["text"] = df["text"].apply(lambda el: el.strip())
df.head()

<>:20: SyntaxWarning: invalid escape sequence '\/'
<>:24: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\/'
<>:24: SyntaxWarning: invalid escape sequence '\w'
C:\Users\hp\AppData\Local\Temp\ipykernel_16156\2108273019.py:20: SyntaxWarning: invalid escape sequence '\/'
  df["text"] = df["text"].str.replace(".+:\/.+t.co\/.+", "")
C:\Users\hp\AppData\Local\Temp\ipykernel_16156\2108273019.py:24: SyntaxWarning: invalid escape sequence '\w'
  df["text"] = df["text"].str.replace("[^\w\s]","")
C:\Users\hp\AppData\Local\Temp\ipykernel_16156\2108273019.py:20: SyntaxWarning: invalid escape sequence '\/'
  df["text"] = df["text"].str.replace(".+:\/.+t.co\/.+", "")
C:\Users\hp\AppData\Local\Temp\ipykernel_16156\2108273019.py:24: SyntaxWarning: invalid escape sequence '\w'
  df["text"] = df["text"].str.replace("[^\w\s]","")


NameError: name 'df' is not defined

In [8]:
X=df['text'].values
y=df['sentiment'].values
# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(y)

NameError: name 'df' is not defined

In [9]:
# Initialize SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Function to calculate sentiment score for a text using SentimentIntensityAnalyzer
def calculate_sentiment(text):
    sentiment_score = sid.polarity_scores(text)['compound']
    return sentiment_score

# Apply sentiment lexicon to the dataset
df['sentiment_score'] = df['text'].apply(calculate_sentiment)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[['text', 'sentiment_score']], df['sentiment'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])

# Combine TF-IDF features with sentiment scores
import scipy.sparse as sp
X_train_combined = sp.hstack((X_train_tfidf, X_train['sentiment_score'].values.reshape(-1, 1)))
X_test_combined = sp.hstack((X_test_tfidf, X_test['sentiment_score'].values.reshape(-1, 1)))
print(y_test)

NameError: name 'df' is not defined

In [10]:
# Train and evaluate Decision Tree models
# J48 Decision Tree
j48_model = DecisionTreeClassifier()  # Default J48 implementation
j48_model.fit(X_train_combined, y_train)
j48_y_pred = j48_model.predict(X_test_combined)
j48_accuracy = accuracy_score(y_test, j48_y_pred)
print(f"J48 Accuracy: {j48_accuracy:.2f}")
positive_mask = (y_test =='Positive')
negative_mask = (y_test == 'Negative')
neutral_mask = (y_test == 'Neutral')

# Calculate accuracy for each sentiment category
accuracy_positive = accuracy_score(y_test[positive_mask], j48_y_pred[positive_mask])
accuracy_negative = accuracy_score(y_test[negative_mask], j48_y_pred[negative_mask])
accuracy_neutral = accuracy_score(y_test[neutral_mask], j48_y_pred[neutral_mask])

print("Accuracy for Positive Sentiments:", accuracy_positive)
print("Accuracy for Negative Sentiments:", accuracy_negative)
print("Accuracy for Neutral Sentiments:", accuracy_neutral)



# BFTree (Best-First Decision Tree)
bftree_model = DecisionTreeClassifier(splitter='best')  # Best-first decision tree
bftree_model.fit(X_train_combined, y_train)
bftree_y_pred = bftree_model.predict(X_test_combined)
bftree_accuracy = accuracy_score(y_test, bftree_y_pred)
print(f"BFTree Accuracy: {bftree_accuracy:.2f}")

positive_mask = (y_test =='Positive')
negative_mask = (y_test == 'Negative')
neutral_mask = (y_test == 'Neutral')

# Calculate accuracy for each sentiment category
accuracy_positive = accuracy_score(y_test[positive_mask], bftree_y_pred[positive_mask])
accuracy_negative = accuracy_score(y_test[negative_mask], bftree_y_pred[negative_mask])
accuracy_neutral = accuracy_score(y_test[neutral_mask], bftree_y_pred[neutral_mask])

print("Accuracy for Positive Sentiments:", accuracy_positive)
print("Accuracy for Negative Sentiments:", accuracy_negative)
print("Accuracy for Neutral Sentiments:", accuracy_neutral)

# OneR Decision Tree
oner_model = DecisionTreeClassifier(max_depth=1)  # OneR decision tree (depth limited to 1)
oner_model.fit(X_train_combined, y_train)
oner_y_pred = oner_model.predict(X_test_combined)
oner_accuracy = accuracy_score(y_test, oner_y_pred)
print(f"OneR Accuracy: {oner_accuracy:.2f}")

positive_mask = (y_test =='Positive')
negative_mask = (y_test == 'Negative')
neutral_mask = (y_test == 'Neutral')

# Calculate accuracy for each sentiment category
accuracy_positive = accuracy_score(y_test[positive_mask], oner_y_pred[positive_mask])
accuracy_negative = accuracy_score(y_test[negative_mask], oner_y_pred[negative_mask])
accuracy_neutral = accuracy_score(y_test[neutral_mask], oner_y_pred[neutral_mask])

print("Accuracy for Positive Sentiments:", accuracy_positive)
print("Accuracy for Negative Sentiments:", accuracy_negative)
print("Accuracy for Neutral Sentiments:", accuracy_neutral)

NameError: name 'X_train_combined' is not defined

In [11]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics import accuracy_score
# from scipy.optimize import minimize

# # Assuming df is your preprocessed DataFrame with 'text' and 'sentiment' columns

# # # Splitting data into features (X) and target labels (y)
# X = df['text']
# y = df['sentiment']

# # Splitting data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Tokenization and TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# # Define the objective function to optimize
# def objective_function(params):
#     # Unpack the parameters
#     n_estimators, max_depth = params
    
#     # Initialize and train the Random Forest model with the current parameters
#     rf_model = RandomForestClassifier(n_estimators=int(n_estimators), max_depth=int(max_depth), random_state=42)
#     rf_model.fit(X_train_tfidf, y_train)
    
#     # Predict sentiment labels for the test set
#     y_pred = rf_model.predict(X_test_tfidf)
    
#     # Calculate accuracy as the objective to maximize
#     accuracy = accuracy_score(y_test, y_pred)
    
#     return -accuracy  # Negative because we want to maximize accuracy

# # Initial guess for the parameters
# initial_guess = [50, 10]  # Example initial guess for n_estimators and max_depth

# # Bounds for the parameters (optional)
# bounds = [(10, 100), (2, 20)]  # Example bounds for n_estimators and max_depth

In [12]:
# # Run the optimization using Nelder-Mead algorithm
# result = minimize(objective_function, initial_guess, bounds=bounds, method='Nelder-Mead')

In [13]:
# # Print the best parameters found
# best_params = result.x
# best_accuracy = -result.fun  # Convert back to positive accuracy
# print("Best Parameters:", best_params)
# print("Best Accuracy:", best_accuracy)

# # Train the final Random Forest model with the best parameters
# best_n_estimators, best_max_depth = best_params
# final_dt_model = DecisionTreeClassifier(max_depth=int(best_max_depth), random_state=42)
# final_dt_model.fit(X_train_tfidf, y_train)

# # Evaluate the final model on the test set
# y_pred_final = final_dt_model.predict(X_test_tfidf)
# final_accuracy = accuracy_score(y_test, y_pred_final)
# print("Final Accuracy:", final_accuracy)


# # final_rf_model = RandomForestClassifier(n_estimators=int(best_n_estimators), max_depth=int(best_max_depth), random_state=42)
# # final_rf_model.fit(X_train_tfidf, y_train)

# # # Evaluate the final model on the test set
# # y_pred_final = final_rf_model.predict(X_test_tfidf)
# # final_accuracy = accuracy_score(y_test, y_pred_final)
# # print("Final Accuracy:", final_accuracy)

In [14]:
# import xgboost as xgb
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Tokenization and TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# # Define the XGBoost model
# xgb_model = xgb.XGBClassifier(
#     objective='multi:relu',  # for multiclass classification
#     num_class=len(np.unique(y)),  # number of classes
#     max_depth=8,  # example depth, tune this parameter
#     learning_rate=0.1,  # example learning rate, tune this parameter
#     n_estimators=500,  # number of trees, tune this parameter
#     random_state=42
# )

# # Train the XGBoost model
# xgb_model.fit(X_train_tfidf, y_train)

# # Predict sentiment labels for the test set
# y_pred = xgb_model.predict(X_test_tfidf)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

def objective_function(params):
    """Objective function to optimize. Takes a list of hyperparameters as input."""
    C, gamma = params
    
    # Initialize the SVM model with the given hyperparameters
    model = SVC(C=C, gamma=gamma)
    
    # Train the model
    model.fit(X_train_preprocessed, y_train)
    
    # Evaluate the model on the validation set
    y_pred = model.predict(X_valid_preprocessed)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_valid, y_pred)
    
    return -accuracy  # Minimize negative accuracy

def tlbo(objective_function, bounds, population_size=10, max_generations=100):
    """Teaching-Learning-Based Optimization (TLBO) algorithm."""
    num_dimensions = len(bounds)
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (population_size, num_dimensions))
    
    for generation in range(max_generations):
        # Evaluate the fitness of each solution
        fitness = np.array([objective_function(individual) for individual in population])
        
        # Find the best solution (teacher)
        teacher_index = np.argmin(fitness)
        teacher = population[teacher_index]
        
        # Teacher phase
        for i in range(population_size):
            population[i] += np.random.uniform(-1, 1, num_dimensions) * (teacher - population[i])
        
        # Learner phase
        for i in range(population_size):
            peer_index = np.random.choice([j for j in range(population_size) if j != i])
            peer = population[peer_index]
            population[i] += np.random.uniform(-1, 1, num_dimensions) * (peer - population[i])
        
        # Clip solutions to the bounds
        population = np.clip(population, bounds[:, 0], bounds[:, 1])
    
    # Return the best solution found
    best_index = np.argmin(fitness)
    best_solution = population[best_index]
    best_fitness = fitness[best_index]
    
    return best_solution, best_fitness

# Assuming df is your preprocessed DataFrame with 'text' and 'sentiment' columns

# Splitting data into features (X) and target labels (y)
X = df['text']
y = df['sentiment']

# Splitting data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Create a preprocessing pipeline
preprocessing_pipeline = make_pipeline(
    TfidfVectorizer(),
    StandardScaler(with_mean=False)  # Assuming sparse input, no need to center
)

# Preprocess the training and validation data
X_train_preprocessed = preprocessing_pipeline.fit_transform(X_train)
X_valid_preprocessed = preprocessing_pipeline.transform(X_valid)

# Define the search space for hyperparameters
search_space = np.array([(0.1, 10),   # C
                         (0.01, 1)])  # gamma

# Run TLBO to optimize hyperparameters
best_params, best_fitness = tlbo(objective_function, search_space)

# Print the best parameters found
print("Best Parameters:", best_params)
print("Best Accuracy:", -best_fitness)  # Convert back to positive accuracy

# Train the final SVM model with the best parameters
C, gamma = best_params
final_model = SVC(C=C, gamma=gamma)
final_model.fit(X_train_preprocessed, y_train)

# Evaluate the final model on the test set
X_test_preprocessed = preprocessing_pipeline.transform(X_test)
y_pred_final = final_model.predict(X_test_preprocessed)
final_accuracy = accuracy_score(y_test, y_pred_final)
print("Final Accuracy:", final_accuracy)


NameError: name 'df' is not defined

In [16]:
# # Initialize SentimentIntensityAnalyzer
# sid = SentimentIntensityAnalyzer()

# # Function to calculate sentiment score for a text using SentimentIntensityAnalyzer
# def calculate_sentiment(text):
#     sentiment_score = sid.polarity_scores(text)['compound']
#     return sentiment_score

# # Apply sentiment lexicon to the dataset
# df['sentiment_score'] = df['text'].apply(calculate_sentiment)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df[['text', 'sentiment_score']], df['sentiment'], test_size=0.2, random_state=42)

# vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
# X_train_tfidf = vectorizer.fit_transform(X_train['text'])
# X_test_tfidf = vectorizer.transform(X_test['text'])

# # Combine TF-IDF features with sentiment scores
# import scipy.sparse as sp
# X_train_combined = sp.hstack((X_train_tfidf, X_train['sentiment_score'].values.reshape(-1, 1)))
# X_test_combined = sp.hstack((X_test_tfidf, X_test['sentiment_score'].values.reshape(-1, 1)))

# # Train and evaluate Decision Tree models
# # J48 Decision Tree
# j48_model = DecisionTreeClassifier()  # Default J48 implementation
# j48_model.fit(X_train_combined, y_train)
# j48_y_pred = j48_model.predict(X_test_combined)
# j48_accuracy = accuracy_score(y_test, j48_y_pred)
# print(f"J48 Accuracy: {j48_accuracy:.2f}")

# # BFTree (Best-First Decision Tree)
# bftree_model = DecisionTreeClassifier(splitter='best')  # Best-first decision tree
# bftree_model.fit(X_train_combined, y_train)
# bftree_y_pred = bftree_model.predict(X_test_combined)
# bftree_accuracy = accuracy_score(y_test, bftree_y_pred)
# print(f"BFTree Accuracy: {bftree_accuracy:.2f}")

# # OneR Decision Tree
# oner_model = DecisionTreeClassifier(max_depth=1)  # OneR decision tree (depth limited to 1)
# oner_model.fit(X_train_combined, y_train)
# oner_y_pred = oner_model.predict(X_test_combined)
# oner_accuracy = accuracy_score(y_test, oner_y_pred)
# print(f"OneR Accuracy: {oner_accuracy:.2f}")